In [1]:
import pandas as pd

In [2]:
all_features = ["tweet_id" , "tweet_timestamp"]
train = pd.read_csv("s3://recsys-challenge-2020/training.tsv", encoding="utf-8",
                    names = all_features, usecols= [2, 8], sep="\x01"
                   )
print('train loaded')

train loaded


In [3]:
train_sorted = train.sort_values(by=['tweet_timestamp']).reset_index()
print('train sorted')
n_head = 90
n_tail = 10
train_set = train_sorted.head(int(len(train_sorted)*(n_head/100)))
val_set = train_sorted.tail(int(len(train_sorted)*(n_tail/100)))

train sorted


In [4]:
train_set_embeddings = pd.read_csv('s3://recsys-challenge-2020/all_lang_output_train.csv')

In [5]:
val_set_embeddings = pd.read_csv('s3://recsys-challenge-2020/all_lang_output_val.csv')

In [6]:
train_embeddings_set = pd.concat([train_set, train_set_embeddings], axis = 1)
val_embeddings_set = pd.concat([val_set.reset_index(), val_set_embeddings], axis =1)
train_embeddings = pd.concat([train_embeddings_set, val_embeddings_set])

In [22]:
val_embeddings_set = pd.concat([val_set.reset_index(), val_set_embeddings], axis =1)
train_embeddings = pd.concat([train_embeddings_set, val_embeddings_set])

In [ ]:
original_train_embeddings = pd.merge(train, train_embeddings, how = 'left', left_on = 'tweet_id', right_on = 'tweet_id')

In [27]:
train_embeddings_index_sorted = train_embeddings.sort_values(['index'])

In [31]:
train_embeddings_index_sorted

,index,tweet_id,tweet_timestamp,Unnamed: 0,sentence_embedding,level_0
76053656,0,D4D1EBDE74F74C5DA529959AF979625C,1581258144,76053656,[-0.02144662 0.20354895 0.00645234 0.040057...,NaN
17033142,1,BFB529DAB6D384EB83E899A72AB3830D,1581018910,17033142,[ 0.04176716 -0.28724265 -0.1052285 -0.072304...,NaN
72408289,2,519078C7834E9642508F72A6C2D0F3B7,1581244866,72408289,[-0.11801001 0.18592697 -0.13860588 -0.248360...,NaN
73681069,3,52AAE9E33EFAC8C478C57B31A9E31ED1,1581250124,73681069,[ 0.3813068 -0.01353616 -0.18093751 -0.339108...,NaN
2832897,4,89C1298C55EB3D68E2784F0BFB69E6F8,1580958805,2832897,[-0.10751743 0.1597824 -0.1880484 -0.032986...,NaN
...,...,...,...,...,...,...
28416522,148075233,21D58EF51BEA1B7F2F99FCC8100600A1,1581073345,28416522,[ 0.0050545 -0.3644939 -0.04321434 -0.012071...,NaN
12175351,148075234,594EB7611FF4DA83CA6DD1BBD1E37735,1581538343,12175351,[-0.39366525 -0.3945667 0.00835489 -0.139878...,145443066.0
9046584,148075235,93DDD9BAA2AA21BBFB487326781811F3,1581525336,9046584,[-0.05744382 -0.05043431 -0.21667148 -0.065127...,142314299.0
45836385,148075236,3EE9B7EFC6C792299B0DDA5B7F408860,1581133279,45836385,[ 0.06306173 -0.2593712 -0.03239084 0.054223...,NaN


In [30]:
train

,tweet_id,tweet_timestamp
0,D4D1EBDE74F74C5DA529959AF979625C,1581258144
1,BFB529DAB6D384EB83E899A72AB3830D,1581018910
2,519078C7834E9642508F72A6C2D0F3B7,1581244866
3,52AAE9E33EFAC8C478C57B31A9E31ED1,1581250124
4,89C1298C55EB3D68E2784F0BFB69E6F8,1580958805
...,...,...
148075233,21D58EF51BEA1B7F2F99FCC8100600A1,1581073345
148075234,594EB7611FF4DA83CA6DD1BBD1E37735,1581538343
148075235,93DDD9BAA2AA21BBFB487326781811F3,1581525336
148075236,3EE9B7EFC6C792299B0DDA5B7F408860,1581133279


In [34]:
train_set

,index,tweet_id,tweet_timestamp
0,19856362,66BB647424F5AD25A562F1C50D688626,1580947200
1,70014084,1BD6E406781F48D3FB023E1212BAD6DC,1580947200
2,45640141,7A6C40F8B6FE18A164C8A426E968867D,1580947200
3,103063682,691021EDFDBED14FDF59FACCDFC4B68B,1580947200
4,54351326,F17D06139541C3EDB831451219B5D69F,1580947200
...,...,...,...
133267709,6993239,F1529D503347DA75EFFD67CC1D9DA0F9,1581486678
133267710,12499592,F1529D503347DA75EFFD67CC1D9DA0F9,1581486678
133267711,100627863,A6EC935E0DD8039D6957A46634F41068,1581486678
133267712,56805658,F1529D503347DA75EFFD67CC1D9DA0F9,1581486678


In [32]:
len(train_embeddings_index_sorted)

148075237

In [33]:
len(train)

148075238

In [37]:
train_sorted.loc[133267713]

index                                      56733728
tweet_id           F1529D503347DA75EFFD67CC1D9DA0F9
tweet_timestamp                          1581486678
Name: 133267713, dtype: object

In [38]:
train_sorted.loc[133267714]

index                                     123786896
tweet_id           F1529D503347DA75EFFD67CC1D9DA0F9
tweet_timestamp                          1581486678
Name: 133267714, dtype: object

In [55]:
desired_line = pd.read_csv('s3://recsys-challenge-2020/training.tsv', skiprows = 123786896, nrows = 1)

In [60]:
desired_line.columns

Index(['101\t17713\t9405\t119152\t119002\t118823\t9531\t10530\t13767\t9665\t118649\t33323\t23466\t9405\t18623\t27023\t58303\t48345\t119\t119\t119\t9665\t118649\t33323\t23466\t11018\t9405\t119152\t9531\t11489\t9580\t118762\t9519\t119285\t130\t14040\t17655\t9580\t31531\t125\t14040\t18382\t68943\t9576\t16605\t58303\t48345\t119\t119\t119\t137\t49866\t27572\t10112\t11259\t12674\t11403\t108\t80426\t11403\t108\t9560\t22333\t14120\t131\t120\t120\t188\t119\t11170\t120\t171\t10731\t48205\t11259\t12396\t11211\t11281\t10174\t11090\t102F2AA2F668B2C6FF79297EB03A516166A\t2414AC78AAA91EC5CC89F15011ED1254F1529D503347DA75EFFD67CC1D9DA0F9Photo\tPhoto\tPhoto\tPhotoTopLevel125C57F4FA6D4E110983FB11B52EFD4E1581486678D75AAE3CCFA44BD84C35942CA4D03E56124594false1578050488AFEA4500BC740BFD6303642AA57DAFF25576false1519362341false'], dtype='object')

In [57]:
desired_line_backup = desired_line.copy()

In [59]:
desired_line.columns = ["text_ tokens", "hashtags", "tweet_id", "present_media", "present_links", "present_domains",\
                "tweet_type","language", "tweet_timestamp", "enaged_with_user_id", "engaged_with_user_follower_count",\
               "engaged_with_user_following_count", "engaged_with_user_is_verified", "engaged_with_user_account_creation",\
               "enaging_user_id", "enaging_user_follower_count", "enaging_user_following_count", "enaging_user_is_verified",\
               "enaging_user_account_creation", "engagee_follows_engager", "reply_timestamp", "retweet_timestamp",\
                "retweet_with_comment_timestamp", "like_timestamp"]

ValueError: Length mismatch: Expected axis has 1 elements, new values have 24 elements

In [39]:
train_sorted.loc[133267715]

index                                      11908278
tweet_id           F1529D503347DA75EFFD67CC1D9DA0F9
tweet_timestamp                          1581486678
Name: 133267715, dtype: object

In [40]:
val_set

,index,tweet_id,tweet_timestamp
133267715,11908278,F1529D503347DA75EFFD67CC1D9DA0F9,1581486678
133267716,54640167,41D145799B12B26F5C580B46D7A0B88E,1581486678
133267717,102671906,94062C22EE1C754B558A39E206B116E4,1581486678
133267718,54937609,D434D803B122A2C7206E0495A955CDC4,1581486678
133267719,38246969,952589011D88FED05B4444CBBD90A172,1581486678
...,...,...,...
148075233,11678309,9A8F193BB63A9C3D3D74E0EBBCD28319,1581551999
148075234,38584247,B9654FC21A888D3D63E30EE2CB35E698,1581551999
148075235,55912866,9A8F193BB63A9C3D3D74E0EBBCD28319,1581551999
148075236,124669351,9A8F193BB63A9C3D3D74E0EBBCD28319,1581551999


In [54]:
train_embeddings_index_sorted[train_embeddings_index_sorted['index'] == 123786896]

,index,tweet_id,tweet_timestamp,Unnamed: 0,sentence_embedding,level_0


,index,tweet_id,tweet_timestamp,Unnamed: 0,sentence_embedding,level_0
76053656,0,D4D1EBDE74F74C5DA529959AF979625C,1581258144,76053656,[-0.02144662 0.20354895 0.00645234 0.040057...,NaN
17033142,1,BFB529DAB6D384EB83E899A72AB3830D,1581018910,17033142,[ 0.04176716 -0.28724265 -0.1052285 -0.072304...,NaN
72408289,2,519078C7834E9642508F72A6C2D0F3B7,1581244866,72408289,[-0.11801001 0.18592697 -0.13860588 -0.248360...,NaN
73681069,3,52AAE9E33EFAC8C478C57B31A9E31ED1,1581250124,73681069,[ 0.3813068 -0.01353616 -0.18093751 -0.339108...,NaN
2832897,4,89C1298C55EB3D68E2784F0BFB69E6F8,1580958805,2832897,[-0.10751743 0.1597824 -0.1880484 -0.032986...,NaN
...,...,...,...,...,...,...
28416522,148075233,21D58EF51BEA1B7F2F99FCC8100600A1,1581073345,28416522,[ 0.0050545 -0.3644939 -0.04321434 -0.012071...,NaN
12175351,148075234,594EB7611FF4DA83CA6DD1BBD1E37735,1581538343,12175351,[-0.39366525 -0.3945667 0.00835489 -0.139878...,145443066.0
9046584,148075235,93DDD9BAA2AA21BBFB487326781811F3,1581525336,9046584,[-0.05744382 -0.05043431 -0.21667148 -0.065127...,142314299.0
45836385,148075236,3EE9B7EFC6C792299B0DDA5B7F408860,1581133279,45836385,[ 0.06306173 -0.2593712 -0.03239084 0.054223...,NaN
